<center><img src="http://sydney.edu.au/images/content/about/logo-mono.jpg"></center>

<center><h1>Statistical Learning and Data Mining (QBUS3820)</h1></center>
<center><h2>Tutorial 13: Artificial Neural Network </h2></center>
<br>


In this tutorial, we will build a spam detector using ANN. To this end, I will build and train a neural network using Keras with a Tensorflow backend. You will learn how to optimise the network using Keras interface to the tensor flow deep learning library. 
In particular, we will apply weight regularization to improve the performance of an deep learning neural network.
Weight regularization not only provide an approach to reduce the overfitting of a deep learning neural network model on the training data. 
But also improve the performance of the model on new data, such as the holdout test set.

Our workflow proceeds with the following steps.

<a href="#Data-Preparation">Data Preparation</a> <br>
<a href="#Model-Specification">Model Specification</a> <br>
<a href="#Model-Compilation">Model Compilation</a> <br>
<a href="#Fitting-the-model">Fitting the model</a> <br>
<a href="#Model-Prediction">Model Prediction</a> <br>
<a href="#Model-Evaluation">Model Evaluation</a> <br>

This notebook relies on the following libraries and settings.

In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
import pandas as pd
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


## Data Preparation

Here I received spam.data which is quite difficult to be read correctly. Hence I used R to split the original data set into train and test sets. Then after importing into python, we will preprocess the data so that we can start building your own neural network!

In [3]:
train_data = pd.read_csv('/Users/leannedong/Desktop/ML-DataMiningCourses/Datasets/spam_train.csv',skiprows=[0],header=None)

In [49]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,58
0,4210,0.00,0.00,1.17,0.0,0.00,0.00,0.0,0.00,0.00,...,0.000,0.284,0.0,0.284,0.213,0.071,1.565,12,72,email
1,4311,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,...,3.024,0.059,0.0,0.059,0.000,0.237,5.016,43,311,email
2,1316,0.27,0.27,0.55,0.0,0.27,0.27,0.0,1.39,0.27,...,0.000,0.279,0.0,2.001,0.093,0.000,3.706,63,341,spam
3,3819,0.00,0.00,0.00,0.0,0.67,0.00,0.0,0.00,0.00,...,0.117,0.117,0.0,0.000,0.000,0.000,1.222,5,33,email
4,2951,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,...,0.000,0.000,0.0,0.000,0.000,0.000,1.250,2,10,email


In [4]:
detect={'spam':1,'email':0}

In [5]:
train_data.iloc[:,58] = [detect[item] for item in train_data.iloc[:,58]]
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,58
0,4210,0.00,0.00,1.17,0.0,0.00,0.00,0.0,0.00,0.00,...,0.000,0.284,0.0,0.284,0.213,0.071,1.565,12,72,0
1,4311,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,...,3.024,0.059,0.0,0.059,0.000,0.237,5.016,43,311,0
2,1316,0.27,0.27,0.55,0.0,0.27,0.27,0.0,1.39,0.27,...,0.000,0.279,0.0,2.001,0.093,0.000,3.706,63,341,1
3,3819,0.00,0.00,0.00,0.0,0.67,0.00,0.0,0.00,0.00,...,0.117,0.117,0.0,0.000,0.000,0.000,1.222,5,33,0
4,2951,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,...,0.000,0.000,0.0,0.000,0.000,0.000,1.250,2,10,0


In [6]:
train_data.to_csv('train_data.csv',header=False, index=False)

In [7]:
test_data = pd.read_csv('/Users/leannedong/Desktop/ML-DataMiningCourses/Datasets/spam_test.csv',skiprows=[0],header=None)

In [11]:
test_data.head()

,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,58
0,3,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,...,0.01,0.143,0.0,0.276,0.184,0.01,9.821,485,2259,spam
1,4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,...,0.00,0.137,0.0,0.137,0.000,0.00,3.537,40,191,spam
2,10,0.06,0.12,0.77,0.0,0.19,0.32,0.38,0.00,0.06,...,0.04,0.030,0.0,0.244,0.081,0.00,1.729,43,749,spam
3,13,0.00,0.69,0.34,0.0,0.34,0.00,0.00,0.00,0.00,...,0.00,0.056,0.0,0.786,0.000,0.00,3.728,61,261,spam
4,21,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.729,0.0,0.729,0.000,0.00,3.833,9,23,spam


In [8]:
test_data.iloc[:,58] = [detect[item] for item in test_data.iloc[:,58]]
test_data.head(5)

,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,58
0,3,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,...,0.01,0.143,0.0,0.276,0.184,0.01,9.821,485,2259,1
1,4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,...,0.00,0.137,0.0,0.137,0.000,0.00,3.537,40,191,1
2,10,0.06,0.12,0.77,0.0,0.19,0.32,0.38,0.00,0.06,...,0.04,0.030,0.0,0.244,0.081,0.00,1.729,43,749,1
3,13,0.00,0.69,0.34,0.0,0.34,0.00,0.00,0.00,0.00,...,0.00,0.056,0.0,0.786,0.000,0.00,3.728,61,261,1
4,21,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.729,0.0,0.729,0.000,0.00,3.833,9,23,1


In [9]:
test_data.to_csv('test_data.csv',header=False, index=False)

In [14]:
type(test_data.head(1))

pandas.core.frame.DataFrame

## Import data

In [10]:
import pandas as pd
import numpy as np
from keras.models import Sequential  # building NN layer by layer
from keras.layers import Dense # randomly initialise weights to small number close to 0
from keras.regularizers import l2 # reduce the risk of overfitting
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix

training_data = pd.read_csv("train_data.csv")
test_data = pd.read_csv("test_data.csv")  
hidden_size = 2
input_size = 58
output_size = 1
x_train = training_data.iloc[:,0:input_size]
y_train = training_data.iloc[:,input_size:input_size + output_size]
x_test = test_data.iloc[:, 0:input_size]
y_test = test_data.iloc[:, input_size:input_size + output_size]


## Model Specification
Now we are ready to build our network with Keras. First, we need to specify the Architecture- How many layers, how many nodes in each layer, what activation functions do you want to use in each layer.

In [11]:
# Model definition
model = Sequential() # no need to put any parameter as we are defining layer manually
model.add(Dense(2, activation='relu', input_shape=(58,), kernel_regularizer=l2(0.1)))
#model.add(Dense(2, activation='relu', input_shape=(58,)))
#model.add(Dense(6, activation='relu'))
#model.add(Dense(6, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


Instructions for updating:
Colocations handled automatically by placer.


Without the regularizer, the accuracy is as low as only 59%.

## Model Compilation

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
# Define early_stopping_monitor
callbacks = [EarlyStopping(monitor='val_loss', patience=2),ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

## Fitting the Model
Here we train our model with our data

In [18]:
model.summary()
# fit model
#history = model.fit(x_train, y_train, batch_size=128, verbose=1, epochs=1000)
model.fit(x_train, y_train,epochs=20,callbacks=callbacks,verbose=1,batch_size=128,validation_data=(x_test, y_test))

# Model Evaluation
eval = model.evaluate(x_test, y_test)
print("\nModel Loss: "+str(eval[0]))
print("Model Accuracy: "+str(eval[1]))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2)                 118       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 3         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________
Train on 3220 samples, validate on 1379 samples
Epoch 1/20
3220/3220 [==============================] - 0s 13us/step - loss: 0.2246 - acc: 0.9146 - val_loss: 0.2344 - val_acc: 0.9137
Epoch 2/20
3220/3220 [==============================] - 0s 12us/step - loss: 0.2225 - acc: 0.9174 - val_loss: 0.2354 - val_acc: 0.9028
Epoch 3/20
3220/3220 [==============================] - 0s 12us/step - loss: 0.2210 - acc: 0.9186 - val_loss: 0.2329 - val_acc: 0.9086
Epoch 4/20
3220/3220 [==============================] - 0s 12us/step - loss: 0.2197 - a

## Model Prediction

In [9]:
# Predicting the Test set results. 
#If the output is > 0.5, then label as spam else no spam
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)

# Creating the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[818   3]
 [  4 554]]
